--- Day 7: Handy Haversacks ---

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

    light red bags contain 1 bright white bag, 2 muted yellow bags.
    dark orange bags contain 3 bright white bags, 4 muted yellow bags.
    bright white bags contain 1 shiny gold bag.
    muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
    shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
    dark olive bags contain 3 faded blue bags, 4 dotted black bags.
    vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
    faded blue bags contain no other bags.
    dotted black bags contain no other bags.

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

    A bright white bag, which can hold your shiny gold bag directly.
    A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
    A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
    A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
    So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [15]:
# add the example data 

import re

def read_file_to_arry():
    file1 = open('data/day7.txt', 'r') 
    Lines = file1.readlines()
    rules = []
    for line in Lines:
        rules.append(line.rstrip('\n'))
    return rules

# parse to a dictionary for O(1) lookup - 

def parse_to_dictionary():
    rules = read_file_to_arry()
    rules_dic = {}
    for rule in rules:
        bags = re.compile(" contain|, ").split(rule)
        bag_list = []
        for bag in bags:
            num = re.findall('\d', bag)
            if (num):
                num = int(num[0])
            else:
                num = 0
            bag_list.append((re.sub('bags|\\.|bag|\d| ', '', bag), num))
        rules_dic[bag_list[0][0]] = bag_list[1:]
    return rules_dic

In [19]:
class Node(object):
    def __init__(self, name, value):
        self.name = name
        self.value = value
        self.children = []
    def add_child(self, obj):
        self.children.append(obj)
    def __str__(self, level=0):
        ret = "\t"*level+repr(self.name)+repr(self.value)+"\n"
        for child in self.children:
            ret += child.__str__(level+1)
        return ret

    def __repr__(self):
        return '<tree node representation>'

def build_tree(node, rules_dic):
    children = rules_dic[node.name]
    if (len(children) == 1 and children[0][0] == 'noother'):
        return node
    else:
        for child in children:
            node.add_child(build_tree(Node(child[0], child[1]), rules_dic))
    return node


In [20]:
def search_tree(node, key, result):
    result = []
    if (node.name == key):
        result.append(1)
    for child in node.children:
        result.extend(search_tree(child, key, result))
    return result

def search_bag(rules_dic):
    count_shiny_gold_bags = 0
    for key in rules_dic:
        if (key != 'shinygold'):
            tree = build_tree(Node(key, 1), rules_dic)
            count = len(search_tree(tree, 'shinygold', []))
            if (count > 0):
                count_shiny_gold_bags += 1
    return count_shiny_gold_bags

rules_dic = parse_to_dictionary()
part_one_answer = search_bag(rules_dic)
print("Part one answer: " + str(part_one_answer))

Part one answer: 226


--- Part Two ---

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

    faded blue bags contain 0 other bags.
    dotted black bags contain 0 other bags.
    vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
    dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

    shiny gold bags contain 2 dark red bags.
    dark red bags contain 2 dark orange bags.
    dark orange bags contain 2 dark yellow bags.
    dark yellow bags contain 2 dark green bags.
    dark green bags contain 2 dark blue bags.
    dark blue bags contain 2 dark violet bags.
    dark violet bags contain no other bags.

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [18]:
def flatten(aList):
    t = []
    for i in aList:
        if not isinstance(i, list):
            t.append(i)
        else:
            t.extend(flatten(i))
    return t

def traverse(node, result):
    children = node.children
    if (children == []):
        return node.value
    else:
        if (node.name != 'shinygold'):
            result.append(node.value)
        for child in node.children:
            out = traverse(child, []) * node.value
            result.append(out)
    return result
    
def search_bag(rules_dic):
    tree = build_tree(Node('shinygold', 1), rules_dic)
    count = sum(flatten(traverse(tree, [])))
    return count
  
rules_dic = parse_to_dictionary()
part_two_answer = search_bag(rules_dic)
print("Part two answer: " + str(part_two_answer))

Part two answer: 9569
